## Imports

In [12]:
import os
import torch
import torchaudio
import pandas as pd
import matplotlib as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import MelSpectrogram


ModuleNotFoundError: No module named 'matplotlib'

## Loading Data

- `train_audio/`: short recordings of individual species with labels - *supervised learning*
- `train_soundscapes/`: long recordings without labels - *unsupervised learning*
- `train.csv`: metadata for the training audio

One thing that can be done is to map species name to label index.

In [8]:
# loading metadata

metadata_csv = "data/train.csv"
train_audio_dir = "data/train_audio"

df = pd.read_csv(metadata_csv)

In [10]:
species_list = sorted(df["primary_label"].unique())
len(species_list)

206